In [34]:
import json, nltk
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline  

In [36]:
df = pd.read_json('semantics.json')
df['content'][0]

'ISABELA, Philippines – The rainy season has just started, but the municipality of Luna in Isabela province in the northern Philippines already declared state of calamity after their health office recorded 54 cases of dengue in just the month of June. Dengue is a fever, usually fatal, caused by a bite of Aedes mosquito. It targets the immune system of the body. Dr Claire Francisco, Luna’s health officer, said on Tuesday, June 30, the number of cases reported for the month is “significantly" higher compared to the numbers reported in the same month in previous years. Although there is no case of death from dengue yet, the increase in the reported cases of the disease prompted the local government unit to declare a state of calamity to be able to release funds for the purchase of medical equipment. Francisco said the purchase of equipment will also be anticipation of more cases during the rainy season. Thirteen out of 19 barangays (villages) in the town were affected, she said. Francisco

In [40]:
text = df['content'][0]
sents = nltk.word_tokenize(text)

In [42]:
words = nltk.pos_tag(sents)
words

[('ISABELA', 'NNP'),
 (',', ','),
 ('Philippines', 'NNP'),
 ('–', 'VBZ'),
 ('The', 'DT'),
 ('rainy', 'NN'),
 ('season', 'NN'),
 ('has', 'VBZ'),
 ('just', 'RB'),
 ('started', 'VBN'),
 (',', ','),
 ('but', 'CC'),
 ('the', 'DT'),
 ('municipality', 'NN'),
 ('of', 'IN'),
 ('Luna', 'NNP'),
 ('in', 'IN'),
 ('Isabela', 'NNP'),
 ('province', 'NN'),
 ('in', 'IN'),
 ('the', 'DT'),
 ('northern', 'JJ'),
 ('Philippines', 'NNPS'),
 ('already', 'RB'),
 ('declared', 'VBD'),
 ('state', 'NN'),
 ('of', 'IN'),
 ('calamity', 'NN'),
 ('after', 'IN'),
 ('their', 'PRP$'),
 ('health', 'NN'),
 ('office', 'NN'),
 ('recorded', 'VBD'),
 ('54', 'CD'),
 ('cases', 'NNS'),
 ('of', 'IN'),
 ('dengue', 'NN'),
 ('in', 'IN'),
 ('just', 'RB'),
 ('the', 'DT'),
 ('month', 'NN'),
 ('of', 'IN'),
 ('June', 'NNP'),
 ('.', '.'),
 ('Dengue', 'NNP'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('fever', 'NN'),
 (',', ','),
 ('usually', 'RB'),
 ('fatal', 'JJ'),
 (',', ','),
 ('caused', 'VBN'),
 ('by', 'IN'),
 ('a', 'DT'),
 ('bite', 'NN'),
 ('of', 